In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
#cur_path = "/content/drive/MyDrive/SNA/final_project/"
cur_path = "/content/drive/MyDrive/Colab Notebooks/final-project/"
os.chdir(cur_path)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/final-project


In [2]:
# %%capture
# # Environment Setup (note that capture silences the console output)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 2.2 MB/s 
     |████████████████████████████████| 3.5 MB 7.1 MB/s 


In [3]:
!python -c "import torch; print(torch.__version__)"

1.11.0+cu113


In [4]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.11.0+cu113.htmlve
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.htmlve
     |████████████████████████████████| 43 kB 215 kB/s 
  Created wheel for torch-cluster: filename=torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl size=309643 sha256=2c85bd8be5284db75c1119afcf6d332c2a04d6eb03500fbd9bcdf04a61873b30
  Stored in directory: /root/.cache/pip/wheels/bc/c7/3e/258dd72b35d7a459264143ad5bfe97b9dc5eef90069ca2f13f
Successfully built torch-cluster
Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 750 kB 6.9 MB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████▌                   | 834.1 MB 1.3 MB/s eta 0:16:22tcmalloc: large alloc 1147494400 bytes == 0x391b8000 @  0x7fbbe96ad615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x

In [ ]:
import random
import time
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
import torch_geometric
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import degree
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
df = df = pd.read_csv("transactions_train.csv.zip", nrows=20000)
df.head()

In [ ]:
# split train and test data
train, test = train_test_split(df.values, test_size=0.3, random_state=1)
train_df = pd.DataFrame(train, columns=df.columns)
test_df = pd.DataFrame(test, columns=df.columns)

In [ ]:
train_df # len

In [ ]:
test_df # len

In [ ]:
# encode IDs to be numeric
customer_codes = pp.LabelEncoder()
article_codes = pp.LabelEncoder()
train_df['customer_id_idx'] = customer_codes.fit_transform(train_df['customer_id'].values)
train_df['article_id_idx'] = article_codes.fit_transform(train_df['article_id'].values)

In [ ]:
# number of unique customers in training set
train_customer_ids = train_df['customer_id'].unique()
len(train_customer_ids)

In [ ]:
# number of unique articles in training set
train_article_ids = train_df['article_id'].unique()
len(train_article_ids)

In [ ]:
print(len(test_df))
# filter test set to only include IDs in the training set
test_df = test_df[(test_df['customer_id'].isin(train_customer_ids)) & (test_df['article_id'].isin(train_article_ids))]
len(test_df)

In [ ]:
# redo label encoder
test_df['customer_id_idx'] = customer_codes.transform(test_df['customer_id'].values)
test_df['article_id_idx'] = article_codes.transform(test_df['article_id'].values)

In [ ]:
n_customers = train_df['customer_id_idx'].nunique()
n_customers

In [ ]:
n_articles = train_df['article_id_idx'].nunique()
n_articles

In [ ]:
def data_loader(data, batch_size, n_cust, n_art):

    def samp_neg(x): # randomly samples if a product was not purchased by a customer
        while True:
            neg_id = random.randint(0, n_art - 1)
            if neg_id not in x:
                return neg_id

    int_items_df = data.groupby('customer_id_idx')['article_id_idx'].apply(list).reset_index()
    indices = [x for x in range(n_cust)] # create indicies for number of customers

    if n_cust < batch_size: # sample for batch size
        customers = [random.choice(indices) for _ in range(batch_size)] # random sample customer
    else:
        customers = random.sample(indices, batch_size)
    customers.sort()
    customers_df = pd.DataFrame(customers,columns = ['customers'])

    int_items_df = pd.merge(int_items_df, customers_df, how = 'right', left_on = 'customer_id_idx', right_on = 'customers')
    pos_items = int_items_df['article_id_idx'].apply(lambda x : random.choice(x)).values # sample if a customer bought an article
    neg_items = int_items_df['article_id_idx'].apply(lambda x: samp_neg(x)).values # sample if a customer did not buy an article

    return (
        torch.LongTensor(list(customers)).to(device), 
        torch.LongTensor(list(pos_items)).to(device) + n_cust, 
        torch.LongTensor(list(neg_items)).to(device) + n_cust
    )

In [ ]:
cust_t = torch.LongTensor(train_df.customer_id_idx) # customer tensor
art_t = torch.LongTensor(train_df.article_id_idx) + n_customers # article tensor 

train_edge_index = torch.stack(( 
  torch.cat([cust_t, art_t]),
  torch.cat([art_t, cust_t])
)).to(device)
train_edge_index # edge index list and reversed

In [ ]:
def compute_loss(customers_emb, pos_emb, neg_emb):
  pos_scores = torch.mul(customers_emb, pos_emb).sum(dim=1) # bought article
  neg_scores = torch.mul(customers_emb, neg_emb).sum(dim=1) # did not buy article
      
  loss = torch.mean(F.softplus(neg_scores - pos_scores,beta=1.5)) # softplus loss, beta =1.5
      
  return loss

In [ ]:
class RecModel(nn.Module):
  def __init__(
      self,
      latent_dim, 
      num_layers,
      num_customers,
      num_articles,
      #model, # 'NGCF' or 'LightGCN'
      dropout=0.1 # Only used in NGCF
  ):
    class NNColabFilter(MessagePassing):
        def __init__(self, latent_dim, dropout, bias=True, **kwargs):  
          super(NNColabFilter, self).__init__(aggr='add', **kwargs)

          self.dropout = dropout

          self.lin_1 = nn.Linear(latent_dim, latent_dim, bias=bias) # layer 1
          self.lin_2 = nn.Linear(latent_dim, latent_dim, bias=bias) # layer 2

          self.init_parameters() # initialize params


        def init_parameters(self): # ramdom sample params
          nn.init.xavier_uniform_(self.lin_1.weight) 
          nn.init.xavier_uniform_(self.lin_2.weight)


        def forward(self, x, edge_index):
          # normalize
          from_, to_ = edge_index
          deg = degree(to_, x.size(0), dtype=x.dtype)
          deg_inv_sqrt = deg.pow(-0.5)
          deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
          norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

          # messages
          out = self.propagate(edge_index, x=(x, x), norm=norm)

          # aggregate
          out += self.lin_1(x)

          # update 
          out = F.dropout(out, self.dropout, self.training)
          return F.softmax(out)

        def message(self, x_j, x_i, norm): # messages
          return norm.view(-1, 1) * (self.lin_1(x_j) + self.lin_2(x_j * x_i))
    
    super(RecModel, self).__init__()
    self.model = NNColabFilter(latent_dim, dropout) # establish model
    self.embedding = nn.Embedding(num_customers + num_articles, latent_dim) # establish input embeddings


    self.convs = nn.ModuleList(NNColabFilter(latent_dim, dropout=dropout) for _ in range(num_layers))

    self.model.init_parameters() # initialize params



  def init_parameters(self): # random sample params
    nn.init.xavier_uniform_(self.embedding.weight, gain=1)
    
  def forward(self, edge_index):
    # message
    emb0 = self.embedding.weight
    embs = [emb0]

    # aggregate
    emb = emb0
    for conv in self.convs:
      emb = conv(x=emb, edge_index=edge_index)
      embs.append(emb)

    # update
    out = (
      torch.cat(embs, dim=-1) )
    
    return emb0, out


  def encode_minibatch(self, customers, pos_items, neg_items, edge_index):
    emb0, out = self(edge_index)
    return (
        out[customers], 
        out[pos_items], 
        out[neg_items], 
        emb0[customers],
        emb0[pos_items],
        emb0[neg_items]
    )

In [ ]:
def get_preds_and_score(customer_Embed_wts, article_Embed_wts, n_customers, n_articles, train_df, test_df, K=12):
  test_customer_ids = torch.LongTensor(train_df.customer_id_idx.unique())
  # relation score
  rel_score = torch.mm(customer_Embed_wts, torch.transpose(article_Embed_wts, 0, 1))
  # purchase interaction between customers and articles
  stacked_interation = torch.stack((torch.LongTensor(train_df.customer_id_idx.values),torch.LongTensor(train_df.article_id_idx.values)))
  # create purchas matrix
  interaction_vector = torch.ones((len(train_df)), dtype=torch.float64)
  # establish fully connected 
  interaction_tensor = torch.sparse.FloatTensor(stacked_interation, interaction_vector, (n_customers, n_articles)).to_dense().to(device)
  # new relation score
  final_rel_score = torch.mul(rel_score, (1-interaction_tensor))
  # most related nodes
  top_rel_indices = torch.topk(final_rel_score, K).indices
  # create reltion df
  top_rel_indices_df = pd.DataFrame(top_rel_indices.cpu().numpy(), columns = ['Top_Article_Index_'+str(x+1) for x in range(0,K)])
  top_rel_indices_df['customer_id'] = top_rel_indices_df.index
  # related article predictions
  top_rel_indices_df['top_rel_articles_preds'] = top_rel_indices_df[['Top_Article_Index_'+str(x+1) for x in range(0,K)]].values.tolist()
  top_rel_indices_df = top_rel_indices_df[['customer_id', 'top_rel_articles_preds']]

  ##### TEST EVALUATION

  # find purchased items 
  test_interacted_items = test_df.groupby('customer_id_idx')['article_id_idx'].apply(list).reset_index()
  # combined preds and actual purchases
  get_preds_df = pd.merge(test_interacted_items, top_rel_indices_df, how = 'left', left_on = 'customer_id_idx', right_on = ['customer_id'])
  # evaluate predictions
  get_preds_df['correct_preds'] = [list(set(a).intersection(b)) for a, b in zip(get_preds_df.article_id_idx, get_preds_df.top_rel_articles_preds)]
  get_preds_df['recall'] = get_preds_df.apply(lambda x: len(x['correct_preds'])/len(x['article_id_idx']), axis = 1)
  get_preds_df['acc'] = get_preds_df.apply(lambda x: len(x['correct_preds'])/len(x), axis=1)

  return get_preds_df.acc.mean(), get_preds_df.recall.mean(), get_preds_df.correct_preds, get_preds_df.top_rel_articles_preds

In [ ]:
latent_dim = 64
n_layers = 2 

EPOCHS = 25
BATCH_SIZE = 1024
DECAY = 0.001
LR = 0.005 
K = 12 # HM challenge allows 12 article recommendations

In [ ]:
def train_and_eval(model, optimizer, train_df):

  recall_list = []
  acc_list = []
  loss_list=[]

  for epoch in tqdm(range(EPOCHS)):
      # calculate number of batches
      n_batch = int(len(train)/BATCH_SIZE)
    
      # train model
      model.train()
      for batch_idx in range(n_batch):

          # zero out the gradient 
          optimizer.zero_grad()

          # data loader to find customer and article purchases
          customers, pos_items, neg_items = data_loader(train_df, BATCH_SIZE, n_customers, n_articles)
          # get embeddings for each node based on purchase interations
          customers_emb, pos_emb, neg_emb, customerEmb0,  posEmb0, negEmb0 = model.encode_minibatch(customers, pos_items, neg_items, train_edge_index)

          # compute loss
          loss = compute_loss(customers_emb, pos_emb, neg_emb)
          # backpropagation
          loss.backward()
          # take step for gradient
          optimizer.step()
          # collect loss
          loss_list.append(loss)

      model.eval() # use trained model
      with torch.no_grad(): # zero out gradient
          _, out = model(train_edge_index)
          final_customer_Embed, final_article_Embed = torch.split(out, (n_customers, n_articles))
          accuracy, recall, correct_preds, preds12 = get_preds_and_score(final_customer_Embed, final_article_Embed, n_customers, n_articles, train_df, test_df, K)

      acc_list.append(round(accuracy,4))
      recall_list.append(round(recall,4))

  return (acc_list, recall_list, correct_preds, preds12, loss_list)

In [ ]:
ngcf = RecModel(
  latent_dim=latent_dim, 
  num_layers=n_layers,
  num_customers=n_customers,
  num_articles=n_articles
)
ngcf.to(device)

optimizer = torch.optim.Adam(ngcf.parameters(), lr=LR)
print("Size of Learnable Embedding : ", [x.shape for x in list(ngcf.parameters())])

In [ ]:
accuracy, recall, correct_preds, preds, train_loss = train_and_eval(ngcf, optimizer, train_df)

In [ ]:
epoch_list = [(i+1) for i in range(EPOCHS*13)]

In [ ]:
train_l = []
for i in train_loss:
  train_l.append(i.cpu().detach().numpy())
train_l=np.squeeze(train_l)

plt.plot(epoch_list, train_l, label='Training Loss')

plt.xlabel('Batches')
plt.ylabel('Loss')
plt.legend()

In [ ]:
epoch_list = [(i+1) for i in range(EPOCHS)]

In [ ]:
plt.plot(epoch_list, recall, label='Recall')
plt.plot(epoch_list, accuracy, label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.legend()

In [ ]:
preds

In [ ]:
correct_preds

In [ ]:
counter = 0
for i in correct_preds:
  counter += len(i)

# final accuracy
counter/len(correct_preds)